In [1]:
# %%
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from collections import Counter

/home/ares/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Model

In [2]:

model_name = "emilyalsentzer/Bio_ClinicalBERT"
max_length = 512

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=5)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

## Loading Data

In [3]:

labels = []
texts = []
with open("./data/medical_data/train.dat", "r") as fh:
    train_lines = fh.readlines()
for line in train_lines:
    splitline = line.split('\t')
    labels.append(splitline[0])
    texts.append(splitline[1])
# %%


In [4]:
# convert to int
labels = [eval(i) for i in labels]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.20)

In [6]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=max_length)

In [7]:
from datasets import Dataset
def preprocessing_function(examples):
    return tokenizer(examples['text'],truncation=True, max_length=max_length,padding='max_length')

train_data  = Dataset.from_dict({'text':X_train,'label':y_train})
train_data =  train_data.map(preprocessing_function)

test_data  = Dataset.from_dict({'text':X_test,'label':y_test})
test_data =  test_data.map(preprocessing_function)
# test_dataset = Dataset.from_dict({'text':X_test,'labels':y_test})


In [8]:
from utils.logging import logger
logger.info(len(test_data['input_ids'][0]))
logger.info(len(test_data['input_ids'][1]))
logger.info(test_data['label'][0])

512
512
5


In [9]:
from datasets import DatasetDict
dev_data = DatasetDict({'train':train_data, 'test':test_data})

In [15]:
#dev_data['train'][0]

In [10]:

# %%
batch_size = 4

args = TrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    report_to="none",
    weight_decay=0.01,
    output_dir='./medical_text/',
    metric_for_best_model='accuracy',)

In [11]:

# %%
#https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}


trainer = Trainer(
    model,
    args,
    train_dataset=dev_data['train'],
    eval_dataset=dev_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)
trainer.train()

# %%


/home/ares/.local/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ares/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nl

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
# import torch 
# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()
# cuda.close()
# torch.cuda.is_available()

True

In [57]:
import gc
gc.collect()

3676